In [1]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append("src")

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import interact

In [14]:
# Supón que importaste desde src/motorcontrol:
from motorcontrol.rotary_motor import RotaryMotor, trapezoidal_profile
from motorcontrol.encoder import Encoder
from motorcontrol.filters import ExponentialFilter
from motorcontrol.speed_controller import SpeedControllerPID, SpeedControllerLogger, plot_speed_pid
from motorcontrol.position_controller import TrapezoidalPlanner, PositionControllerPID, PositionControllerLogger, plot_position_pid

def run_position_pid_simulation(kp, ki, kd):
    planner = TrapezoidalPlanner(pos0=0.0, posf=90.0, vmax=20.0, amax=10.0)
    pos_pid = PositionControllerPID(kp=kp, ki=ki, kd=kd)

    motor = RotaryMotor(
        inertia=0.02,
        friction_viscous=0.05,   # Sube un poco para amortiguar
        max_torque=1.6,
        friction_coulomb=0.02,
        external_torque=0.01
    )
    encoder = Encoder(motor, noise_std_deg=0.2)
    velocity_filter = ExponentialFilter(alpha=0.4)

    speed_pid = SpeedControllerPID(
        motor, encoder, velocity_filter,
        kp=0.2, ki=0.003, kd=0.002,
        integral_limit=50.0,
        deadband=0.1
    )

    dt = 0.01
    t_max = planner.Tf + 2.0
    steps = int(t_max / dt)

    logger = PositionControllerLogger()

    for step in range(steps):
        t = step * dt
        pos_ref, _ = planner.get_ref(t)
        pos_real = encoder.read_position_deg()

        vel_pid_status = pos_pid.update(pos_ref, pos_real, dt)
        vel_ref = vel_pid_status["vel_ref"]

        speed_pid_status = speed_pid.update(vel_ref, dt)
        motor.update(dt)

        logger.log(t, pos_ref, pos_real, vel_pid_status, speed_pid_status)

    plot_position_pid(logger)
    
interact(
    run_position_pid_simulation,
    kp=widgets.FloatSlider(value=2.0, min=0.0, max=10.0, step=0.01),
    ki=widgets.FloatSlider(value=0.0, min=0.0, max=2.0, step=0.05),
    kd=widgets.FloatSlider(value=0.0, min=0.0, max=2.0, step=0.05)
);


interactive(children=(FloatSlider(value=2.0, description='kp', max=10.0, step=0.01), FloatSlider(value=0.0, de…